In [1]:
import json
import pandas as pd
import os

In [10]:
def json_to_dataframe(json_directory):
    data_table = []

    for file_name in os.listdir(json_directory):
        if file_name.endswith('.json'):
            file_path = os.path.join(json_directory, file_name)
            
            with open(file_path, 'r') as file:
                value = json.load(file)
            
 
            if 'data' in value:
                row_data = {
                    'id': value['id'],
                    'operation': value['op'],
                    'timestamp': value['ts'],
                    **value['data']
                }
            elif 'set' in value:
                row_data = {
                    'id': value['id'],
                    'operation': value['op'],
                    'timestamp': value['ts'],
                }
                row_data.update(value['set']) 
            
            data_table.append(row_data)


    data_frame = pd.DataFrame(data_table)
    
    return data_frame

In [11]:
df_accounts = json_to_dataframe('/Users/pinjammodal3/Downloads/dwh-coding-challenge/data/accounts')
df_cards = json_to_dataframe('/Users/pinjammodal3/Downloads/dwh-coding-challenge/data/cards')
df_savingacc = json_to_dataframe('/Users/pinjammodal3/Downloads/dwh-coding-challenge/data/savings_accounts')

In [22]:
# 1.Visualize the complete historical table view of each tables in tabular format in stdout 
print('Accounts:')
print(df_accounts)
print('Cards:')
print(df_cards)
print('Saving Accounts:')
print(df_savingacc)

Accounts:
           id operation      timestamp savings_account_id card_id  \
0  a1globalid         u  1577890800000                sa1     NaN   
1  a1globalid         u  1577926800000                NaN      c1   
2  a1globalid         u  1579163400000                NaN      c2   
3  a1globalid         u  1579078860000                NaN           
4  a1globalid         u  1577865600000                NaN     NaN   
5  a1globalid         c  1577863800000                NaN     NaN   
6  a1globalid         u  1577894400000                NaN     NaN   

  phone_number account_id     name   address                    email  
0          NaN        NaN      NaN       NaN                      NaN  
1          NaN        NaN      NaN       NaN                      NaN  
2          NaN        NaN      NaN       NaN                      NaN  
3          NaN        NaN      NaN       NaN                      NaN  
4     87654321        NaN      NaN       NaN                      NaN  
5    

In [57]:
import sqlite3
conn = sqlite3.connect(':memory:')  # Creates an in-memory SQLite database
df_accounts.to_sql('accounts', conn, index=False)
df_cards.to_sql('cards', conn, index=False)
df_savingacc.to_sql('saving_accounts', conn, index=False)

8

In [86]:
# 2. Visualize the complete historical table view of the denormalized joined table in stdout by joining these three tables
print('Historical Accounts & Cards')
query = '''with source1 as (SELECT accounts.timestamp at,cards.timestamp ct,accounts.*,cards.*
        FROM accounts
        left join cards on cards.timestamp=accounts.timestamp order by accounts.timestamp asc),
        source2 as (SELECT accounts.timestamp at,cards.timestamp ct,accounts.*,cards.*
        FROM cards
        left join accounts on cards.timestamp=accounts.timestamp order by accounts.timestamp asc),
        source as (select * from source1 union all select * from source2)
        select distinct *
        from source
'''
result_df = pd.read_sql_query(query, conn)
print(result_df)
print('Historical Accounts & Saving Accounts')
query2 = '''with source1 as (SELECT accounts.timestamp at,saving_accounts.timestamp ct,accounts.*,saving_accounts.*
        FROM accounts
        left join saving_accounts on saving_accounts.timestamp=accounts.timestamp order by accounts.timestamp asc),
        source2 as (SELECT accounts.timestamp at,saving_accounts.timestamp ct,accounts.*,saving_accounts.*
        FROM saving_accounts
        left join accounts on saving_accounts.timestamp=accounts.timestamp order by accounts.timestamp asc),
        source as (select * from source1 union all select * from source2)
        select distinct *
        from source
'''
result_df2 = pd.read_sql_query(query2, conn)
print(result_df2)


Historical Accounts & Cards
              at            ct          id operation     timestamp  \
0   1.577864e+12           NaN  a1globalid         c  1.577864e+12   
1   1.577866e+12           NaN  a1globalid         u  1.577866e+12   
2   1.577891e+12           NaN  a1globalid         u  1.577891e+12   
3   1.577894e+12           NaN  a1globalid         u  1.577894e+12   
4   1.577927e+12  1.577927e+12  a1globalid         u  1.577927e+12   
5   1.579079e+12           NaN  a1globalid         u  1.579079e+12   
6   1.579163e+12  1.579163e+12  a1globalid         u  1.579163e+12   
7            NaN  1.579079e+12        None      None           NaN   
8            NaN  1.578654e+12        None      None           NaN   
9            NaN  1.578159e+12        None      None           NaN   
10           NaN  1.579298e+12        None      None           NaN   
11           NaN  1.579361e+12        None      None           NaN   
12           NaN  1.578314e+12        None      None          

In [99]:
# 3.From result from point no 2, discuss how many transactions has been made, when did each of them occur, and how much the value of each transaction?  
selected_rows_df = result_df.loc[result_df['credit_used'] > 0, ['at', 'ct', 'operation:1', 'credit_used']]
print("total credit used:")
print(selected_rows_df)
# 3.From result from point no 2, discuss how many transactions has been made, when did each of them occur, and how much the value of each transaction?  
selected_rows_df2 = result_df2.loc[result_df2['balance'] > 0, ['at', 'ct', 'operation:1', 'balance']]
print("\n total balance change:")
print(selected_rows_df2)

total credit used:
    at            ct operation:1  credit_used
11 NaN  1.579361e+12           u      37000.0
12 NaN  1.578314e+12           u      12000.0
13 NaN  1.578420e+12           u      19000.0

 total balance change:
    at            ct operation:1  balance
9  NaN  1.578649e+12           u  40000.0
10 NaN  1.578654e+12           u  21000.0
11 NaN  1.579505e+12           u  33000.0
12 NaN  1.577956e+12           u  15000.0
